<a href="https://colab.research.google.com/github/shaharvac95/Capstone-project-2024-generating-multi-scale-graphs/blob/main/SapShell_Engine_Cloud_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase
!pip install requests beautifulsoup4

In [ ]:
# @title Default title text
from IPython.display import HTML, display, Javascript, clear_output
from firebase import firebase
from google.colab import output
import json, time
import requests
import re
from collections import Counter
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import math
import datetime
import pytz
from nltk.stem import PorterStemmer
from pickle import EMPTY_SET



user_name=""
# Initialize Firebase
FBconn = firebase.FirebaseApplication('https://sapshell-cloud-default-rtdb.europe-west1.firebasedatabase.app/Sapshell', None)
firebase = firebase.FirebaseApplication('https://sapshell-cloud-default-rtdb.europe-west1.firebasedatabase.app/', None)

# Retrieve data from Firebase
results = list(firebase.get('/Sapshell/Index', None).keys())


searching_html="""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell</title>
    <style>
        body {
                  font-family: Arial, sans-serif;
                  margin: 0;
                  padding: 0;
                  background-color: #0c2d40;
                  text-align: center;
                  height: 100vh;
              }
        .header {
            background-color: #0c2d40;
            padding:20 px;
            text-align: center;
            color: white;
        }
        .search-bar-container {
            display: flex;
            justify-content: center;
            padding: 20px;
        }
         .search-bar-description-container {
            display: flex;
            justify-content: center;
            width: 100%; /* Ensures the container takes the full width */
        }
        .search-bar-description {
            color: #0c2d40;
            background-color: white;
            margin-bottom: 20px;
            padding: 15px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
            width: 50%; /* Adjust width as needed */
            text-align: center;
        }
        .search-bar {
            width: 50%;
            padding: 10px;
            border: none;
            border-radius: 5px;
            font-size: 16px;
        }
        .search-button {
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
        }
        .results-container {
            width: 60%;
            border-collapse: collapse;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        .result-item {
            background-color: white;
            margin-bottom: 10px;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            display: none; /* Hide advanced search initially */

        }
        .result-item h3 {
            color: #007bff;
            margin: 0;
        }
        .result-item p {
            color: #333;
        }
        .result-item .url {
            color: #006621;
        }
        .sidebar {
            width: 200px;
            background-color: #e6e8ed;
            color: #0c2d40;
            height: 100vh;
            position: fixed;
            left: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }
       .sidebar h2, .sidebar a, .sidebar p {
            margin-top: 10px;
            margin-bottom: 20px;
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block;
        }
        .sidebar a:hover, .sidebar p:hover {
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }
        .sidebar a, .sidebar p {
            text-decoration: none;
            color: #0c2d40;
        }
    </style>

</head>
<body>
<div class="sidebar">
            <div>
                <h2>Navigation</h2>
                <a id="searchHistoryButton">🔍 User Search History</a>
                <a class="sign-out" onclick="signOut()">🚪 Sign out</a>
            </div>
</div>
    <div class="header">
        <div class="logo">
          <img  width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
        </div>
    <h1 style="color: white">SapShell Search Engine</h1>   </div>
    <div class="search-bar-description-container">
    <div class="search-bar-description">
              We break down your query into words and check each word in our index list, each word in the query that is in the index receives a bonus, at the end we return results in which the largest amount of words appear in the index plus the bonus.
          </div>
          </div>
        <div class="search-bar-container">
          <input type="text" id="query" class="search-bar" placeholder="Search...">
          <button onclick="search()" class="search-button">Search</button>
        </div>
    </div>
<script>
  document.getElementById('searchHistoryButton').addEventListener('click', function() {
       google.colab.kernel.invokeFunction('notebook.display_search_history', [user], {})
  });

  function signOut() {
        // Show an alert message to the user
        localStorage.removeItem("username");
        localStorage.clear();
        alert("Signing out...");
        // Redirect to the homepage after showing the alert
        google.colab.kernel.invokeFunction('notebook.Redirect4', [], {}).then(function(response){}).catch(function(error){console.error(error);});
  }
  function search() {
        let query = document.getElementById('query').value;
        var user = localStorage.getItem("username"); // Inject the Python value of user into JavaScript
        google.colab.kernel.invokeFunction('notebook.search_result', [user,query], {});
  }
</script>

</body>
</html>
"""




# Function to add index
def add_index_in_db(new_word):
    operation =  f"""// Function to update the operation content
    function updateOperation() {{
      // Here you should define what operation you want to display
      var operationDiv = document.getElementById("operation");
      operationDiv.innerText = "Adding/Editing your index, it may take a while.";
    }}

    // Call updateOperation to set the initial value
    updateOperation();"""
    clear_output(wait=True)
    display(HTML(html_loading))
    display(Javascript(operation))
    # Trim leading or trailing whitespaces
    new_word = new_word.strip()
    new_word = new_word.lower()

    # Check if the word is empty
    if not new_word:
        javascript_alert = f"""alert("Error: The word is empty.");"""
        display(Javascript(javascript_alert))
        redirect_to_index_manager()
        return

    # Check if the word contains Unicode characters
    if bool(re.search('[^\x00-\x7F]', new_word)):
        javascript_alert = f"""alert("Error: The word contains Unicode characters.");"""
        display(Javascript(javascript_alert))
        redirect_to_index_manager()
        return

    # Check if the word is in English
    if not new_word.isascii():
       javascript_alert = f"""alert("Error: The word is not in English.");"""
       display(Javascript(javascript_alert))
       redirect_to_index_manager()
       return

    # Fetch the index from the database
    index = firebase.get('/Sapshell/Index', None)

    # Check if the word already exists in the index
    if new_word in index:
       javascript_alert = f"""alert("Message: The word '{new_word}' already exists in the database.");"""
       display(Javascript(javascript_alert))
       redirect_to_index_manager()

    # Fetch all URLs data
    all_urls_data = firebase.get('/Sapshell/URL_indexer', None)

    # Index the new word
    index = index_word_in_db(new_word)
    print(f"Index: {index}")

    as_inserted = firebase.get('/Sapshell/Index/' + new_word, None)

    # Check if the word could be added to the database
    if as_inserted is not None:
       javascript_alert = f"""alert("The word '{new_word}' been added to the database.");"""
       display(Javascript(javascript_alert))
       redirect_to_index_manager()
    else:
      javascript_alert = f"""alert("The word '{new_word}' couldn't be added to the database.");"""
      display(Javascript(javascript_alert))
      redirect_to_index_manager()
      return

    # Store the updated index in the database
    store_index(index)

    # Calculate Rank for the new word
    compute_Rank_for_word(len(all_urls_data), new_word)


def index_word_in_db(new_word):

    try:
        # Fetch the index from the database
        index = firebase.get('/Sapshell/Index', None)

        # Fetch all URLs data
        all_urls_data = firebase.get('/Sapshell/URL_indexer', None)

        # Initialize an empty dictionary for the new word's URL ID list
        url_id_list = {}

        for url_id, url in enumerate(all_urls_data[:300]):
            soup = fetch_page(url)
            if soup is not None:  # Only process if soup is not None
                # Extract words from webpage content
                words = re.findall(r'\b\w+\b', soup.get_text().lower())
                total_words = len(words)
                word_counts = Counter(words)

                # Calculate term frequency for the new word
                new_word = new_word.lower()  # Convert word to lowercase
                tf = 0  # Initialize tf to 0
                if new_word in word_counts:
                    tf = word_counts[new_word] / total_words
                    # Add the URL ID and its TF to the URL ID list
                    url_id_list[f"id:{url_id}"] = {"TF": tf}
                else:
                  continue

        # Store the URL ID list in the index
        if url_id_list:  # Check if url_id_list is not empty
            if new_word in index:
                # Update the URL ID list for the word in the index
                firebase.put('/Sapshell/Index/'+ new_word, 'url_id_list', url_id_list)
            else:
                # Add the new word and its URL ID list to the index
                firebase.put('/Sapshell/Index/', new_word, {'url_id_list': url_id_list})
        return index
    except Exception as e:
        print(f"An error occurred: {e}")


def compute_Rank_for_word(total_documents,new_word):

        new_word = new_word.lower()
        url_id_list = firebase.get('/Sapshell/Index/'+ new_word + '/url_id_list/', None)
        print( url_id_list)
        # Check if url_id_list is not None
        if url_id_list is not None:
            # Check if url_id_list is a dictionary
            if isinstance(url_id_list, dict):
                # Calculate IDF
                idf = math.log(total_documents / len(url_id_list))

                # Iterate over each URL ID and its corresponding TF dictionary
                for url_id, tf_dict in url_id_list.items():
                    # Check if 'TF' is in tf_dict
                    if 'TF' in tf_dict:
                        # Calculate Rank (TF-IDF)
                        tf = tf_dict['TF']
                        rank = tf * idf
                        # Update the TF dictionary with Rank
                        tf_dict['Rank'] = rank

                        # Store the updated TF dictionary back into the Firebase database
                        firebase.put('/Sapshell/Index/'+ new_word + '/url_id_list/', url_id, tf_dict)
            else:
                print(f"Warning: URL ID list for word '{new_word}' is not a dictionary.")
        else:
            print(f"Warning: URL ID list for word '{new_word}' is None.")

def store_index(index):
    if index is not None:
        print(index.items())
        for word, data in index.items():
            result = firebase.put('/Sapshell/Index', word, data)

# Function to delete index
def delete_index(index_to_delete):
    firebase.delete('/Sapshell/Index', index_to_delete)

# Function to perform search
def fetch_indexes_from_firebase():
    """Fetches index data from Firebase."""
    try:
        results = firebase.get('/Sapshell/Index', None)
        if results:
            return list(results.keys())
        else:
            return []
    except Exception as e:
        print(f"Error fetching indexes from Firebase: {e}")
        return []


# Fetch and prepare index data
results = fetch_indexes_from_firebase()
index_list = json.dumps(results)
index_to_delete = None
query = None

firebase_url = 'https://sapshell-cloud-default-rtdb.europe-west1.firebasedatabase.app/'

def fetch_and_process_data_for_chart():
    results = list(firebase.get('/Sapshell/Index', None).keys())
    word_counts = {}
    for word in results:
        word_counts[word] = word_counts.get(word, 0) + 1
        query_results = firebase.get(f'/Sapshell/Index/{word}/url_id_list', None)
        if query_results and isinstance(query_results, dict):
            word_counts[word] = len(query_results.keys())
    rare_words = {word: count for word, count in word_counts.items() if count == 1}
    sorted_rare_words = sorted(rare_words.items(), key=lambda x: x[1], reverse=True)[:6]
    labels = [item[0] for item in sorted_rare_words]
    values = [item[1] for item in sorted_rare_words]
    return json.dumps({"labels": labels, "values": values})

def fetch_indexes_with_most_urls():
    indexes_data = firebase.get('/Sapshell/Index', None)
    if not indexes_data:
        return json.dumps({"labels": [], "values": []})
    url_counts = {}
    for index, data in indexes_data.items():
        if 'url_id_list' in data:
            url_counts[index] = len(data['url_id_list'])
    sorted_url_counts = sorted(url_counts.items(), key=lambda x: x[1], reverse=True)[:5]
    labels = [item[0] for item in sorted_url_counts]
    values = [item[1] for item in sorted_url_counts]
    return json.dumps({"labels": labels, "values": values})

# Fetch data
labels_json = fetch_and_process_data_for_chart()
top_indexes_data = fetch_indexes_with_most_urls()

# Pass the data to JavaScript
display(HTML(f"<script>window.labelsJson = {labels_json}; window.topIndexesData = {top_indexes_data};</script>"))

def redirect_to_results():
    """Redirects to the search results interface."""
    # Display the HTML content for search results
    clear_output(wait=True)
    display(HTML(searching_html))

def redirect_to_index_manager():
    """Redirects to the index manager interface."""
    # Display the HTML content for index manager
    clear_output(wait=True)
    display(HTML(index_manager_html))
    display(Javascript(javascript_code))

def redirect_to_statistics():
    """Redirects to the statistics interface."""
    # Display the HTML content for statistics
    clear_output(wait=True)
    display(HTML(statistics_html))

def redirect_to_homepage():
    """Redirects to the statistics interface."""
    # Display the HTML content for statistics
    clear_output(wait=True)
    display(HTML(html_code_initial))
def redirectToLogin():
  """Redirects to the registration/login interface."""
  # Display the HTML content for registration/login
  clear_output(wait=True)
  display(HTML(register_html))


def display_search_history(username):
    user = username
    print("USERNAME")
    search_history = show_search_history(username)
    if search_history:
        # HTML structure with placeholders for dynamic content
        history_html = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>User Search History</title>
            <style>
              body {{
                  font-family: Arial, sans-serif;
                  margin: 0;
                  padding: 0;
                  background-color: #0c2d40;
                  text-align: center;
                  height: 100vh;
              }}
              .sidebar {{
                  width: 200px;
                  background-color: #e6e8ed;
                  color: #0c2d40;
                  height: 100vh;
                  position: fixed;
                  left: 0;
                  padding: 20px;
                  display: flex;
                  flex-direction: column;
                  justify-content: space-between;
              }}
              .sidebar h2, .sidebar a, .sidebar p {{
                  margin-top: 10px;
                  margin-bottom: 20px;
                  padding: 10px;
                  border-radius: 5px;
                  transition: background-color 0.3s ease;
                  display: block;
              }}
              .sidebar a:hover, .sidebar p:hover {{
                  background-color: #0c2d40;
                  color: #e6e8ed;
                  cursor: pointer;
              }}
              .sidebar a, .sidebar p {{
                  text-decoration: none;
                  color: #0c2d40;
              }}
              .main-container {{
                  margin-left: 200px;
                  width: calc(100% - 200px);
                  display: flex;
                  flex-direction: column;
                  align-items: center;
                  justify-content: flex-start;
                  padding: 20px;
              }}
              .logo {{
                  position: fixed;
                  right: 0;
                  top: 0;
                  padding: 10px;
              }}
              .logo img {{
                  width: 200px;
                  height: 100px;
              }}
              h1 {{
                  color: white;
                  margin: 20px 0;
                  width: 100%;
              }}
              table {{
                  color: white;
                  width: 80%;
                  background-color: #123456; /* Darker shade for the background */
                  border-radius: 5px;
                  padding: 20px;
                  box-shadow: 0 4px 8px rgba(0,0,0,0.3); /* Adding some shadow for depth */
                  overflow-y: auto;
                  max-height: 500px; /* Limit height and enable scrolling */
              }}
              th, td {{
                  padding: 10px;
              }}
              th {{
                  background-color: #0c2d40;
              }}
              tr:nth-child(even) {{
                  background-color: #0c2d40; /* Dark shade for even rows */
              }}
              tr:nth-child(odd) {{
                  background-color: #123456; /* Light shade for odd rows */
              }}
            </style>
        </head>
        <body>
<div class="sidebar">
            <div>
                <h2>Navigation</h2>
                <a id="searchButton">🔍 Search</a>
                <a class="sign-out" onclick="signOut()">🚪 Sign out</a>
            </div>
</div>
          <div class="main-container">
        <div class="logo">
            <img src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
        </div>
            <h1>Search History for {username}</h1>
            <table>
                <thead>
                    <tr>
                        <th>Search Item</th>
                        <th>Time</th>
                    </tr>
                </thead>
                <tbody>
        """

        # Iterate over search history and populate table rows
        count = 0
        for item in reversed(search_history):
          if(count<15):
            count += 1
            print(item)
            history_html += f"""
                    <tr>
                        <td>{item['search_item']}</td>
                        <td>{item['time']}</td>
                    </tr>
            """

        history_html += """
                </tbody>
            </table>
          </div>
          <script>
        document.getElementById('searchButton').addEventListener('click', function() {
       google.colab.kernel.invokeFunction('notebook.Redirect1', [], {})
});

         function signOut() {
        // Show an alert message to the user
        localStorage.removeItem("username");
        localStorage.clear();
        sessionStorage.clear();
        alert("Signing out...");
        // Redirect to the homepage after showing the alert
        google.colab.kernel.invokeFunction('notebook.Redirect4', [], {}).then(function(response){}).catch(function(error){console.error(error);});

  }

          </script>
        </body>
        </html>
        """
        clear_output(wait=True)
        display(HTML(history_html))
    else:
        js_code = 'alert("No search history for user.")'
        display(Javascript(js_code))

def loginuser(username, password):
    users = FBconn.get('/Sapshell/users', None)
    admins = FBconn.get('/Sapshell/Admins', None)
    for user_id in users:
        user = users[user_id]
        if user['Username'] == username and user['Password'] == password:
            user_name = username
            js_code = 'alert("Login successful.")'
            display(Javascript(js_code))
            display(HTML("<style>.form-container{display:none;}</style>"))
            clear_output(wait=True)
            display(HTML(html_code_initial_user))
            return True
        elif username in admins and admins[username]['password'] == password:
          js_code = 'alert("Login successful as Admin.")'
          display(Javascript(js_code))
          display(HTML("<style>.form-container{display:none;}</style>"))
          clear_output(wait=True)
          user_name=username
          display(HTML(html_code_initial_admin))
          return True
    js_code = 'alert("Invalid username or password")'
    display(Javascript(js_code))
    return False



# Retrieve data from Firebase
results = list(firebase.get('/Sapshell/Index', None).keys())

def show_search_history(username):
    print(f"Username_search_history: {username}")
    users = FBconn.get('/Sapshell/users', None)
    for user_id in users:
        user = users[user_id]
        if user['Username'] == username:
            search_history = user.get('Search_History', {})
            return list(search_history)
    #print("Username not found.")
    return None



# Pass the data to JavaScript
#display(HTML(f"<script>window.search_history_json = {search_history_json}"))




def create_account(username, password):
    # Check if username and password are not empty or NULL
    if not username or not password:
        print("Username and password cannot be empty.")
        return

    # Check if username and password lengths are within limits
    if len(username) > 12 or len(password) > 12:
        print("Username and password lengths cannot exceed 12 characters.")
        return

    # Check if username already exists
    if not user_exists(username):
        data = {"Username": username, "Password": password, "Search_History": []}
        result = FBconn.post('/Sapshell/users', data)
        print("Account created successfully.")
        time.sleep(3)
        display(HTML("<style>.form-container{display:none;}</style>"))
        clear_output(wait=True)
        display(HTML(html_code_initial))
    else:
        print("Username already exists.")


def user_exists(username):
      users = FBconn.get('/Sapshell/users', None)
      if users is not None:
          for user_id in users:
              user = users[user_id]
              if user['Username'] == username:
                  return True
      return False


output.register_callback('notebook.create_account', create_account)
# Retrieve data from Firebase
results_manager = list(firebase.get('/Sapshell/Index', None).keys())

# Extract index list
index_list_manager = json.dumps(results_manager)
# Function to perform search

# Function to delete index
def delete_index(index_to_delete):
    firebase.delete('/Sapshell/Index', index_to_delete)

def fetch_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
            'Referer': 'https://www.google.com/',
            'Connection': 'keep-alive'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        else:
 #           print("Error: Failed to fetch webpage. Status code:", response.status_code)
            return None
    except Exception as e:
 #       print("Error: Failed to fetch webpage:", e)
        return None



url_list = firebase.get('/Sapshell/URL_indexer', None)
query_results = []

result_items_html = """
                <div class="result-item">
                  <a href="{url_list[key]}" class="url">{url_list[key]}</a>
                  <h3>title {key}</h3>
                  <p>description {key}</p>
               </div>
        """
# Define the number of results per page
results_per_page = 5
page_number = 2
#iris
# Assuming `page_number` is the current page number and starts from 1

def pagination(number):
    operation =  f"""// Function to update the operation content
    function updateOperation() {{
      // Here you should define what operation you want to display
      var operationDiv = document.getElementById("operation");
      operationDiv.innerText = "Looking for more results, it may take a while.";
    }}

    // Call updateOperation to set the initial value
    updateOperation();"""
    global page_number  # Declare page_number as global so changes inside the function affect the global variable
    if number == "0" and page_number > 1:
        page_number -= 1
    if number == "1":
        page_number += 1
    clear_output(wait=True)
    display(HTML(html_loading))
    display(Javascript(operation))
    bulid_search_page()



html_loading = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Loading Screen</title>
<style>
        body {{
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background-color: #0c2d40
        }}
  .loading-container {{
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    height: 100vh;
  }}
  .loading-text {{
    font-size: 24px;
  }}
  .loader {{
    border: 8px solid #f3f3f3; /* Light grey */
    border-top: 8px solid #3498db; /* Blue */
    border-radius: 50%;
    width: 50px;
    height: 50px;
    animation: spin 1s linear infinite;
  }}
  @keyframes spin {{
    0% {{ transform: rotate(0deg); }}
    100% {{ transform: rotate(360deg); }}
  }}
          .header {{
            background-color: #0c2d40;
            padding:20 px;
            text-align: center;
            color: white;
        }}
</style>
</head>
<body>
<div class="header">
<div class="loading-container">
  <img  width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
  <div id="operation" class="loading-text"></div>
  <div id="funny-sentence" class="loading-text"></div>
  <div class="loader"></div>
</div>
</div>
<script>
    // Array of funny sentences
    var funnySentences = [
      "Why don't scientists trust atoms? Because they make up everything!",
      "Parallel lines have so much in common. It's a shame they'll never meet.",
      "Why did the scarecrow win an award? Because he was outstanding in his field!",
      "I'm reading a book on anti-gravity. It's impossible to put down!",
      "Did you hear about the claustrophobic astronaut? He just needed a little space.",
      "Why don't we write with broken pencils? Because it’s pointless.",
      "I have a joke about construction, but I'm still working on it.",
      "What do you call fake spaghetti? An impasta!"
    ];

    // Function to pick a random sentence
    function pickRandomSentence() {{
      var randomIndex = Math.floor(Math.random() * funnySentences.length);
      return funnySentences[randomIndex];
    }}

    // Function to update the displayed sentence
    function updateSentence() {{
      var funnySentenceDiv = document.getElementById("funny-sentence");
      funnySentenceDiv.innerText = pickRandomSentence();
    }}

    // Update sentence every 7 seconds
    setInterval(updateSentence, 7000);

    // Initial sentence update
    updateSentence();
</script>


</body>
</html>"""

def ranked_results(query):
    stemmer = PorterStemmer()

    # Remove punctuation from the query and stem the words
    query = re.sub(r'[^\w\s]', '', query)
    words = query.lower().split()
    words = [stemmer.stem(word) for word in words]
    index = firebase.get('/Sapshell/Index', None)

    # Fetch matching documents
    matching_documents = {}
    url_counts = Counter()
    for word in words:
        if word in index:
            for url_id, data in index[word]['url_id_list'].items():
                if url_id not in matching_documents:
                    matching_documents[url_id] = {"Rank": 0, "urls": set()}
                matching_documents[url_id]["Rank"] += data["Rank"]
                matching_documents[url_id]["urls"].add(url_id)
                url_counts[url_id] += 1
    print(matching_documents)
    if  matching_documents is not EMPTY_SET:
      # Add bonus to URLs that contain two or more indices
      for url_id, count in url_counts.items():
          if count >= 2:
              matching_documents[url_id]["Rank"] += 0.5 * (count - 1)  # Add 0.5 for each additional index

      # Rank documents
      ranked_documents = sorted(matching_documents.items(), key=lambda x: x[1]["Rank"], reverse=True)
      # Return top documents
      return [url.replace('id:', '') for doc in ranked_documents for url in doc[1]["urls"]]
    else:
      print("Your Search did not match any document.")
    return None

def add_search_history(username, search_item):
    users = FBconn.get('/Sapshell/users', None)
    for user_id in users:
        user = users[user_id]
        if user['Username'] == username:
            history = user.get('Search_History', None)
            # If history is not a list, make it an empty list
            if not isinstance(history, list):
                history = []
            # Include the current date and time with each search item
            jerusalem_tz = pytz.timezone('Asia/Jerusalem')
            search_item_with_time = {
                'search_item': search_item,
                'time': datetime.datetime.now(jerusalem_tz).strftime("%Y-%m-%d %H:%M:%S")
            }
            history.append(search_item_with_time)
            FBconn.put('/Sapshell/users/'+ user_id, "Search_History", history)
            print("Search history updated.")


def bulid_search_page():
  global query_results
  results_html = """"""
  start_index = (page_number - 1) * results_per_page
  end_index = min(start_index + results_per_page, len(query_results))
  try:
    for id in query_results[start_index:end_index]:
      soup = fetch_page(url_list[int(id)])
      if soup is not None:
       if soup.find("meta", attrs={"name": "description"}) is not None:
          results_html += f"""
            <div class="result-item">
              <a href="{url_list[int(id)]}" target="_blank" </a>
              <h3>{soup.title.string}</h3>
              <p>{soup.find("meta", attrs={"name": "description"})["content"]}</p>
            </div>
            """
      else:
        results_html += f"""
          <div class="result-item">
            <a href="{url_list[int(id)]}" target="_blank"</a>
            <h3>{soup.title.string}</h3>
            <p>No description found</p>
          </div>
        """
  except Exception as e:
      print(f"Error accessing Firebase database: {e}")

  html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>SapShell</title>
        <style>
        body {{
                  font-family: Arial, sans-serif;
                  margin: 0;
                  padding: 0;
                  background-color: #0c2d40;
                  text-align: center;
                  height: 100vh;
              }}
        .header {{
            background-color: #0c2d40;
            padding:20 px;
            text-align: center;
            color: white;
        }}
        .search-bar-container {{
            display: flex;
            justify-content: center;
            padding: 20px;
        }}
        .search-bar {{
            max-width: 50%;
            padding: 10px;
            border: none;
            border-radius: 5px;
            font-size: 16px;
        }}
        .search-button {{
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
        }}
    .results-container {{
     display: grid;
     grid-template-columns: 1fr;
     gap: 20px;
     padding: 20px;
     max-width: 1000px;
     margin: 0 auto;
}}

    .result-item {{
    background-color: #f9f9f9;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    transition: box-shadow 0.3s ease;
}}

        .result-item h3 {{
            color: #007bff;
            margin: 0;
        }}
        .result-item p {{
            color: #333;
        }}
        .result-item .url {{
            color: #006621;
        }}
        #prevBtn,
#nextBtn {{
    padding: 8px 20px;
    background-color: #4CAF50;
    color: white;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}}

#prevBtn:hover,
#nextBtn:hover {{
    background-color: #45a049;
}}
        .sidebar {{
            width: 200px;
            background-color: #e6e8ed;
            color: #0c2d40;
            height: 100vh;
            position: fixed;
            left: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }}
       .sidebar h2, .sidebar a, .sidebar p {{
            margin-top: 10px;
            margin-bottom: 20px;
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block;
        }}
        .sidebar a:hover, .sidebar p:hover {{
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }}
        .sidebar a, .sidebar p {{
            text-decoration: none;
            color: #0c2d40;
        }}
    </style>
    </head>
    <body>
<div class="sidebar">
            <div>
                <h2>Navigation</h2>
                <a id="searchHistoryButton">🔍 User Search History</a>
                <a class="sign-out" onclick="signOut()">🚪 Sign out</a>
            </div>
</div>
    <div class="header">
        <div class="logo">
          <img  width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
        </div>
    <h1 style="color: white">SapShell Search Engine</h1>   </div>
        <div class="search-bar-container">
          <input type="text" id="query" class="search-bar" placeholder="Search...">
          <button onclick="search()" class="search-button">Search</button>
        </div>

<div>
        <button id="prevBtn" onclick="prev()">Previous</button>
        <button id="nextBtn" onclick="next()">Next</button>
</div>
          <div class="results-container">
          <!-- Results will be dynamically generated here -->

      </div>

    </div>

    <script>
      document.getElementById('searchHistoryButton').addEventListener('click', function() {{
       google.colab.kernel.invokeFunction('notebook.display_search_history', [user], {{}})
  }});

 function signOut() {{
        // Show an alert message to the user
        localStorage.removeItem("username");
        localStorage.clear();
        sessionStorage.clear();
        alert("Signing out...");
        // Redirect to the homepage after showing the alert
        google.colab.kernel.invokeFunction('notebook.Redirect4', [], {{}}).then(function(response){{}}).catch(function(error){{console.error(error);}});

  }}
  function search() {{
    let query = document.getElementById('query').value;
    var user = localStorage.getItem("username"); // Inject the Python value of `user` into JavaScript
    google.colab.kernel.invokeFunction('notebook.search_result', [user,query], {{}});

}}

  function prev() {{
    let number = "0"
    google.colab.kernel.invokeFunction('notebook.pagination', [number], {{}});
}}

  function next() {{
    let number = "1"
    google.colab.kernel.invokeFunction('notebook.pagination', [number], {{}});
}}
    </script>
    </body>
    </html>
    """
    # Inject the dynamically generated result items HTML into the main HTML content
  html_content = html_content.replace("<!-- Results will be dynamically generated here -->", results_html)
  clear_output(wait=True)
  display(HTML(html_content))

# Function to perform search
def search_index(username,query):
    operation =  f"""// Function to update the operation content
    function updateOperation() {{
      // Here you should define what operation you want to display
      var operationDiv = document.getElementById("operation");
      operationDiv.innerText = "Searching for your query, it may take a while.";
    }}

    // Call updateOperation to set the initial value
    updateOperation();"""
    if not query:
        print("Query is empty.")
        return

    add_search_history(username, query)
    # Assuming `firebase` is initialized properly before this function call
    clear_output(wait=True)
    display(HTML(html_loading))
    display(Javascript(operation))

    global query_results
    query_results = ranked_results(query)
    print(query_results)
    if query_results:
      bulid_search_page()
    else:
      js_code = f"""alert("Sorry, we couldn't find anything related to {query}.")"""
      clear_output(wait=True)
      display(HTML(searching_html))
      display(Javascript(js_code))





index_manager_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background-color: #0c2d40;
        }}
        .search-container {{
            text-align: center;
        }}
        .search-input {{
            width: 60%;
            padding: 10px;
            font-size: 16px;
            margin-bottom: 20px;
            border-radius: 10px;
        }}
        .search-button {{
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
        }}
        .search-button:hover {{
            background-color: #0056b3;
        }}
        table {{
            border-collapse: collapse;
            width: 60%;
            margin: auto;
        }}
        th, td {{
            border: 1px solid #ddd;
            padding: 8px;
            text-align: center;
        }}
        th {{
            background-color: #f2f2f2;
        }}
        .table-wrapper {{
    display: flex;
    justify-content: space-between;
    align-items: center;
}}

#prevBtn,
#nextBtn {{
    padding: 8px 20px;
    background-color: #4CAF50;
    color: white;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}}

#prevBtn:hover,
#nextBtn:hover {{
    background-color: #45a049;
}}
    .sidebar {{
            width: 200px;
            background-color: #e6e8ed;
            color: #0c2d40;
            height: 100vh;
            position: fixed;
            left: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }}
       .sidebar h2, .sidebar a, .sidebar p {{
            margin-top: 10px;
            margin-bottom: 20px;
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block;
        }}
        .sidebar a:hover, .sidebar p:hover {{
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }}
        .sidebar a, .sidebar p {{
            text-decoration: none;
            color: #0c2d40;
        }}
    </style>
</head>
<body>
 <div class="sidebar">
    <div>
        <h2>Navigation</h2>
        <ul>
            <li><a onclick="searchRedirect2()">Index Manager</a></li>
            <li><a onclick="searchRedirect3()">📊 Analytics</a></li>
            <div>
                <p class="sign-out" onclick="signOut()">🚪 Sign out</p>
            </div>
        </ul>
    </div>
</div>
<div class="search-container">
    <div class="logo">
        <img width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
    </div>
    <div>
    <h1 style="color: white">SapShell Index Manager</h1>
        <input type="text" id="query" class="search-input" placeholder="Enter index...">
        <button onClick="search()" type="button" class="search-button">Search</button>
        <button onClick="add_index()" type="button" class="search-button">Add</button>
    </div>
    <div class="table-wrapper">
        <button id="prevBtn" onclick="generateTable(false)";>Previous</button>
        <table id="data-table">
            <thead>
                <tr>
                    <th>Index</th>
                    <th>Actions</th>
                </tr>
            </thead>
            <tbody id="table-body">
                <!-- Data rows will be added dynamically using JavaScript -->
            </tbody>
        </table>
        <button id="nextBtn" onclick="generateTable(true)">Next</button>
    </div>
</div>

</body>
</html>
"""

javascript_code = f"""
  let i = 0
  let data = {index_list};

    function signOut() {{
        // Show an alert message to the user
        localStorage.removeItem("username");
        localStorage.clear();
        alert("Signing out...");
        // Redirect to the homepage after showing the alert
        google.colab.kernel.invokeFunction('notebook.Redirect4', [], {{}}).then(function(response){{}}).catch(function(error){{console.error(error);}});
  }}

  function searchRedirect1() {{
            google.colab.kernel.invokeFunction('notebook.Redirect1', [], {{}}).then(function(response){{}}).catch(function(error){{console.error(error);}});
        }}
  function searchRedirect2() {{
            google.colab.kernel.invokeFunction('notebook.Redirect2', [], {{}}).then(function(response){{}}).catch(function(error){{console.error(error);}});
        }}
  function searchRedirect3() {{
            google.colab.kernel.invokeFunction('notebook.Redirect3', [], {{}}).then(function(response){{}}).catch(function(error){{console.error(error);}});
        }}

  function add_index(query=document.getElementById('query').value) {{
    document.getElementById('query').innerHTML = '';
    google.colab.kernel.invokeFunction('notebook.add_index_in_db', [query], {{}});
}}

function deleteIndex(index) {{
    google.colab.kernel.invokeFunction('notebook.delete_index', [index], {{}});
}}


  function generateTable(next){{
    let tableBody = document.getElementById("table-body");
    tableBody.innerHTML = '';

    if (next) {{
      i += 15; // Move to the next set of indices
      }}
    else {{
            i -= 15; // Move to the previous set of indices
            if (i < 0) {{
                i = 0; // Ensure i doesn't go below 0
            }}
            if (i > data.length) {{
                i = data.length - 1; // Ensure i doesn't go below 0
            }}
    }}
    for (let j = i; j < i + 15 && j < data.length; ++j) {{
        let row = tableBody.insertRow();
        let cell = row.insertCell();
        cell.innerText = data[j];
        cell.style.backgroundColor = "#f2f2f2";

        let buttons = row.insertCell();
        buttons.style.backgroundColor = "#f2f2f2";

        // Create the edit button
        // Create the edit button
        let editButton = document.createElement("button");
        editButton.innerText = "\u270E";
        editButton.style.backgroundColor = "cyan";
        editButton.style.opacity = 0.85;
        editButton.addEventListener("click", function() {{
          let currentCell = row.cells[0]; // Get the cell in the same row
          let cellData = currentCell.innerText; // Get the current cell's data

          // Create an input field
          let inputField = document.createElement("input");
          inputField.type = "text";
          inputField.value = cellData; // Set initial value of the input field to cellData

          // Style the input field
          inputField.style.width = "min-content"; // Make input field fill the entire cell

          // Replace the cell's content with the input field
          currentCell.innerHTML = ''; // Clear existing content of the cell
          currentCell.appendChild(inputField); // Append the input field to the cell

          let saveButton = document.createElement("button");
          saveButton.innerText = "Save";
          saveButton.style.backgroundColor = "green";
          saveButton.style.opacity = 0.85;
          saveButton.addEventListener("click", function() {{
           // Save the edited data and update the cell content
            let newData = inputField.value;
            if(newData != data[j]){{
              deleteIndex(data[j]);
              currentCell.innerText = newData;
              add_index(newData);}}
            else{{
               currentCell.innerText = data[j];
            }}


    }});

       // Append save button to the buttons cell
       cell.appendChild(saveButton);
}});

        // Append the button to the cell
        buttons.appendChild(editButton);


        // Create the delete button
        let deleteButton = document.createElement("button");
        deleteButton.innerText = "✖";
        deleteButton.style.backgroundColor = "red";
        deleteButton.style.opacity = 0.85;
        deleteButton.addEventListener("click", function() {{
            let indexToDelete = data[j];
            deleteIndex(indexToDelete);
            row.remove();
        }});

        // Append the button to the cell
        buttons.appendChild(deleteButton);
    }}
}}


function search() {{
    let query = document.getElementById('query').value;
    if(query == ""){{
      data = {index_list}
        generateTable(false);}}
    else{{
          data = data.filter(item => item.toLowerCase().includes(query));
          generateTable(false);}}




}}


    generateTable(false);
"""







statistics_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.9.4/Chart.js"></script>
    <style>
      body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #0c2d40;
            text-align: center;
            height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
        }}
    .sidebar {{
    width: 200px;
    background-color: #e6e8ed;
    color: #0c2d40;
    height: 100vh; /* Full height of the viewport */
    position: fixed;
    left: 0;
    top: 0; /* Start at the top of the viewport */
    bottom: 0; /* Extend to the bottom of the viewport */
    padding: 20px;
    display: flex;
    flex-direction: column;
    justify-content: space-between;
    z-index: 1000; /* Ensure sidebar is above other content */
}}
       .sidebar h2, .sidebar a, .sidebar p {{
            margin-top: 10px;
            margin-bottom: 20px;
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block;
        }}
        .sidebar a:hover, .sidebar p:hover {{
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }}
        .sidebar a, .sidebar p {{
            text-decoration: none;
            color: #0c2d40;
        }}
        .main-container {{
            margin-left: 200px; /* Adjusted to prevent overlap with sidebar */
            width: calc(100% - 220px); /* Adjust based on sidebar width */
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: flex-start;
            padding: 20px;
        }}
        .logo {{
            position: fixed;
            right: 0;
            top: 0;
            padding: 10px;
        }}
        .logo img {{
            width: 200px;
            height: 100px;
        }}
        .charts-container {{
            display: flex;
            justify-content: center; /* Center charts */
            align-items: center;
            width: 100%;
            flex-wrap: wrap; /* Allow charts to wrap if space is insufficient */
        }}
        .chart {{
            margin: 30px;
            width: 600px; /* Increased width */
            height: 400px; /* Increased height */
            display: block;
        }}
        h1 {{
            color: white;
            margin: 20px 0;
            width: 100%; /* Ensure the header spans the width of its container */
        }}
        .dashboard-summary {{
            color: white;
            margin-bottom: 20px;
        }}
        canvas {{
            background-color: #fff;
            width: 100%; /* Ensure canvas fills chart container */
            height: 100%;
        }}
    </style>
</head>
<body>
    <div class="sidebar">
        <div>
            <h2>Navigation</h2>
            <p><a onclick="searchRedirect2()">Index Manager</a></p>
            <p>📊 Analytics</p>
        </div>
        <div>
            <p class="sign-out" onclick="signOut()">🚪 Sign out</p>
        </div>
    </div>
    <div class="main-container">

        <h1>SapShell Analytics Dashboard</h1>

        <div class="logo">
            <img src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
        </div>
    <div class="main-container">
        <!-- Main container content omitted for brevity -->
        <div id="labelsTableContainer">
            <div class="charts-container">
             <div class="chart" style="display:block;">
                <canvas id="rareWordsChart"></canvas>
            </div>
            <div class="chart" style="display:block;"> <!-- Ensure this div is displayed -->
                <canvas id="topUrlsChart"></canvas> <!-- Corrected ID for the second chart -->
            </div>
        </div>
    </div>
        </div>
    </div>

    <script>


    let rareWordsData = JSON.parse('{labels_json}');
    let topUrlsData = JSON.parse('{top_indexes_data}');
let rareWordsCtx = document.getElementById('rareWordsChart').getContext('2d');

        new Chart(rareWordsCtx, {{
            type: 'bar',
            data: {{
                labels: rareWordsData.labels,
                datasets: [{{
                    label: 'Rare Words Distribution',
                    data: rareWordsData.values,
                    backgroundColor: 'rgba(54, 162, 235, 0.2)',
                    borderColor: 'rgba(54, 162, 235, 1)',
                    borderWidth: 1
                }}]
            }},
            options: {{
                scales: {{
                    yAxes: [{{
                        ticks: {{
                            beginAtZero: true
                        }}
                    }}]
                }}
            }}
        }});

        // Create Top URLs Distribution Chart
        let topUrlsCtx = document.getElementById('topUrlsChart').getContext('2d');
        new Chart(topUrlsCtx, {{
            type: 'bar',
            data: {{
                labels: topUrlsData.labels,
                datasets: [{{
                    label: 'Top URLs Distribution',
                    data: topUrlsData.values,
                    backgroundColor: 'rgba(255, 99, 132, 0.2)',
                    borderColor: 'rgba(255, 99, 132, 1)',
                    borderWidth: 1
                }}]
            }},
            options: {{
                scales: {{
                    yAxes: [{{
                        ticks: {{
                            beginAtZero: true
                        }}
                    }}]
                }}
            }}
        }});

    </script>
</body>
</html>
"""






# Initial HTML content display
html_code_initial = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            height: 100vh;
            margin: 0;
            background-color: #0c2d40;
        }
        .search-container {
            text-align: center;
        }
      .sidebar {
            width: 200px;
            background-color: #e6e8ed;
            color: #0c2d40;
            height: 100vh;
            position: fixed;
            left: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }
       .sidebar h2, .sidebar a, .sidebar p {
            margin-top: 10px;
            margin-bottom: 20px; /* Increased bottom margin for more space */
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block; /* Ensure full width for easier clicking */
        }
        .sidebar a:hover, .sidebar p:hover {
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }
        .sidebar a, .sidebar p {
            text-decoration: none;
            color: #0c2d40;
        }
         .sign-in-btn {
    position: fixed;
    top: 40px; /* Changed from bottom to top */
    right: 20px;
    padding: 10px 20px;
    background-color: #e6e8ed;
    color: #0c2d40;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}
.sign-in-btn:hover {
    background-color: #0c2d40;
    color: #e6e8ed;
}
 .chatbot {
        position: fixed;
        bottom: 20px;
        right: 10px; /* Adjusted from 20px to 10px */
        padding: 15px 30px;
        background-color: #007bff;
        color: #ffffff;
        border: none;
        border-radius: 5px;
        cursor: pointer;
        transition: background-color 0.3s ease, box-shadow 0.3s ease;
        font-weight: bold;
        font-size: 18px;
        text-shadow: 1px 1px 2px #0005;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2);
    }

    .chatbot:hover {
        background-color: #0056b3;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);
    }

    .chatbot-container {
        position: fixed;
        bottom: 20px;
       left: 1700px; /* Adjusted from 20px to 10px */
        width: 300px;
        border: 1px solid #ddd;
        border-radius: 5px;
        background-color: #fff;
        box-shadow: 0 2px 10px rgba(0,0,0,0.2);
        display: none;
    }


        .chat-header {
            background-color: #007bff;
            color: #fff;
            padding: 10px;
            border-radius: 5px 5px 0 0;
        }
        .chat-messages {
            height: 200px;
            overflow-y: auto;
            padding: 10px;
            background: #f9f9f9;
        }
        .chat-input {
            padding: 10px;
            border-top: 1px solid #ddd;
            background: #fff;
        }
        .chat-input input {
            width: calc(100% - 50px);
            padding: 5px;
            margin-right: 5px;
            border: 1px solid #ccc;
            border-radius: 3px;
        }
        .chat-input button {
            width: 40px;
            background: #007bff;
            color: white;
            border: none;
            border-radius: 3px;
            cursor: pointer;
        }
    </style>
</head>
<body>

<div class="search-container">
    <div class="logo">
        <img width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
    </div>
    <h1 style="color: white">Welcome to SapShell</h1>

<button id="signinbutton" class="sign-in-btn">Sign In</button>


<div class="chatbot-container" id="chatbot">
    <div class="chat-header">Chatbot</div>
    <div class="chat-messages" id="chat-messages"></div>
    <div class="chat-input">
        <input type="text" id="chat-input-text" placeholder="Ask me something...">
        <button onclick="sendMessage()">Send</button>
    </div>
</div>
<button class="chatbot" onclick="toggleChatbot()">Chat with us</button>
</div>
<script>
 function toggleChatbot() {
            var chatbot = document.getElementById("chatbot");
            chatbot.style.display = chatbot.style.display === "none" ? "block" : "none";
        }

        function sendMessage() {
            var input = document.getElementById("chat-input-text");
            var messageContainer = document.getElementById("chat-messages");
            var message = document.createElement("div");
            message.textContent = "You: " + input.value;
            messageContainer.appendChild(message);

            // Example of a simple response
            var reply = document.createElement("div");
            reply.textContent = "Bot:  - " + input.value;
            messageContainer.appendChild(reply);
            input.value = ""; // Clear input field

            messageContainer.scrollTop = messageContainer.scrollHeight; // Scroll to the bottom
        }
  document.getElementById("signinbutton").addEventListener("click", function() {
            redirectToLogin();
        });

        // Define redirectToLogin function
        function redirectToLogin() {
            // Trigger the Python callback to display the login/register page
            google.colab.kernel.invokeFunction('notebook.redirectToLogin', [], {}).then(function(response){}).catch(function(error){console.error(error);});
        };
function toggleChatbot() {
    var chatbot = document.getElementById("chatbot");
    chatbot.style.display = chatbot.style.display === "none" ? "block" : "none";
}

function sendMessage() {
    var input = document.getElementById("chat-input-text");
    var messageContainer = document.getElementById("chat-messages");
    var userMessage = document.createElement("div");
    userMessage.textContent = "You: " + input.value;
    messageContainer.appendChild(userMessage);

    var botReply = simulateResponse(input.value); // Get the bot's reply
    var reply = document.createElement("div");
    reply.textContent = "Bot: " + botReply;
    messageContainer.appendChild(reply);

    input.value = ""; // Clear input field
    messageContainer.scrollTop = messageContainer.scrollHeight; // Scroll to the bottom
}

    function simulateResponse(input) {
        // Basic greeting responses
        if (/hi|hello|hey/i.test(input)) {
            return "Hello! 😊 Welcome to SAP help chat!";
        } else if (/how are you?/i.test(input)) {
            return "I'm just a bot, but I'm buzzing along nicely today! 🐝";
        } else if (/name?/i.test(input)) {
            return "I'm SAP Bot. Your go-to assistant for SAP wonders! 🚀";
        }

        // Humorous responses
        else if (/tell me a joke/i.test(input)) {
            return "How does a computer get drunk? It takes screenshots. 🍻";
        } else if (/purpose of life/i.test(input)) {
            return "42, according to my calculations. 🌌 But let's focus on SAP for now!";
        }

        // SAP-related responses
        else if (/sap/i.test(input)) {
            return "SAP stands for Systems, Applications, and Products in Data Processing. It's a world leader in enterprise applications! 🌐";
        } else if (/erp/i.test(input)) {
            return "ERP stands for Enterprise Resource Planning. SAP ERP is a suite of applications that helps businesses run smoothly! 🏢";
        } else if (/hana/i.test(input)) {
            return "SAP HANA is an in-memory database and application development platform for processing high volumes of data in real time. It's like a supercomputer for your data! 🖥";
        } else if (/fiori/i.test(input)) {
            return "SAP Fiori is a design language and user experience approach developed by SAP for use in business applications. It's like a makeover for your software! 💅";
        }
        else if (/abap/i.test(input)) {
        return "ABAP (Advanced Business Application Programming) is a high-level programming language created by SAP. It's the muscle that moves SAP software! 💪";
        } else if (/sapui5/i.test(input)) {
        return "SAPUI5 is a JavaScript framework for building web applications on top of SAP NetWeaver. It's like the paintbrush for creating beautiful SAP apps! 🖌";
        } else if (/gui/i.test(input)) {
        return "SAP GUI (Graphical User Interface) is a tool that provides a user interface to access SAP applications. It's like the front door to your SAP system! 🚪";
        } else if (/netweaver/i.test(input)) {
        return "SAP NetWeaver is a technology platform that allows organizations to integrate data, business processes, elements and more from a variety of sources into unified SAP environments. It's like the glue that holds your SAP landscape together! 🧩";
        } else if (/successfactors/i.test(input)) {
        return "SAP SuccessFactors is a cloud-based human resources software system. It's like a personal assistant for managing your workforce! 👥";
        } else if (/hybris/i.test(input)) {
         return "SAP Hybris is a family of product from a German company Hybris, which sells e-Commerce, Marketing, Sales, Service and Product Content Management Software. It's like a one-stop shop for your commerce needs! 🛍";
        }

        // Catch-all for unrecognized input
        else {
            const randomReplies = [
                "Hmm, I'm not quite sure how to answer that. 🤔 Let's talk SAP!",
                "I think I missed that class in bot school. 😅 Ask me about SAP!",
                "Intriguing question! 🧐 But can we pivot back to SAP?",
                "I'd love to chat more about that, but my expertise is in SAP. 📚",
                "That's outside my cloud of knowledge. 💭 Ask me something SAP-related!"
            ];
            // Return a random response from the catch-all replies
            return randomReplies[Math.floor(Math.random() * randomReplies.length)];
        }

    }


    </script>
</body>
</html>
"""



register_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell Login/Register</title>
    <style>
    body {
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background-color: #0c2d40;
        }

        .main-container {
            margin-left: 200px;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: flex-start;
            height: auto;
            padding: 30px;
        }
        .logo {
            position: fixed;
            right: 0;
            top: 0;
            padding: 10px;
        }
        .logo img {
            width: 200px;
            height: 100px;
        }
        .form-container {
            background-color: #e6e8ed;
            padding: 40px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            width: 300px;
        }
        .form-container h2 {
            color: #0c2d40;
        }
        .input-group {
            margin-bottom: 15px;
        }
        .input-group label {
            display: block;
            text-align: left;
            margin: 5px 0;
        }
        .input-group input {
            width: 100%;
            padding: 10px;
            margin: 5px 0;
            border-radius: 5px;
            border: 1px solid #ddd;
        }
        .button {
            width: 100%;
            padding: 10px;
            border: none;
            border-radius: 5px;
            background-color: #0c2d40;
            color: white;
            cursor: pointer;
            margin-top: 10px;
        }
        .toggle-form {
            background: none;
            color: #0c2d40;
            border: none;
            margin-top: 10px;
            cursor: pointer;
        }
    </style>
</head>
<body>
    <div class="form-container">
    <div class="logo">
        <img width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
    </div>
        <h2>Login</h2>
        <div class="input-group">
            <label for="login-User Name">User Name:</label>
            <input type="email" id="login-email" required>
        </div>
        <div class="input-group">
            <label for="login-password">Password:</label>
            <input type="password" id="login-password" required>
        </div>
        <button class="button" id="login-button">Log In</button>
        <button class="toggle-form" id="show-register-form">Or register</button>
    </div>
    <div class="form-container" style="display: none;">
        <h2>Register</h2>
        <div class="input-group">
            <label for="register-email">User Name:</label>
            <input type="email" id="register-email" required>
        </div>
        <div class="input-group">
            <label for="register-password">Password:</label>
            <input type="password" id="register-password" required>
        </div>

        <button class="button" id="register-button">Register</button>
        <button class="toggle-form" id="show-login-form">Or log in</button>
    </div>

    <script>
document.getElementById('login-button').addEventListener('click', function() {
    var username = document.getElementById('login-email').value;
    var password = document.getElementById('login-password').value;

    google.colab.kernel.invokeFunction('notebook.loginuser', [username, password], {})

    localStorage.setItem("username", username)
});
document.getElementById('register-button').addEventListener('click', function() {
    var username = document.getElementById('register-email').value;
    var password = document.getElementById('register-password').value;
    google.colab.kernel.invokeFunction('notebook.create_account', [username, password], {});
    localStorage.setItem("username", username)
});
 var loginForm = document.querySelectorAll('.form-container')[0];
        var registerForm = document.querySelectorAll('.form-container')[1];
        var showRegisterFormButton = document.getElementById('show-register-form');
        var showLoginFormButton = document.getElementById('show-login-form');

        showRegisterFormButton.addEventListener('click', () => {
            loginForm.style.display = 'none';
            registerForm.style.display = 'block';
        });

        showLoginFormButton.addEventListener('click', () => {
            registerForm.style.display = 'none';
            loginForm.style.display = 'block';
        });

    </script>
</body>
</html>

  """

html_code_initial_admin = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            height: 100vh;
            margin: 0;
            background-color: #0c2d40;
        }
        .search-container {
            text-align: center;
        }
      .sidebar {
            width: 200px;
            background-color: #e6e8ed;
            color: #0c2d40;
            height: 100vh;
            position: fixed;
            left: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }
       .sidebar h2, .sidebar a, .sidebar p {
            margin-top: 10px;
            margin-bottom: 20px; /* Increased bottom margin for more space */
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block; /* Ensure full width for easier clicking */
        }
        .sidebar a:hover, .sidebar p:hover {
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }
        .sidebar a, .sidebar p {
            text-decoration: none;
            color: #0c2d40;
        }
         .sign-in-btn {
            position: fixed;
            bottom: 20px;
            right: 20px;
            padding: 10px 20px;
            background-color: #e6e8ed;
            color: #0c2d40;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }
        .sign-in-btn:hover {
            background-color: #0c2d40;
            color: #e6e8ed;
        }
    </style>
</head>
<body>
<div class="sidebar">
    <div>
        <h2>Navigation</h2>
        <ul>
            <li><a onclick="searchRedirect2()">Index Manager</a></li>
            <li><a onclick="searchRedirect3()">📊 Analytics</a></li>
            <div>
                <p class="sign-out" onclick="signOut()">🚪 Sign out</p>
            </div>
        </ul>
    </div>
</div>
<div class="search-container">
    <div class="logo">
        <img width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
    </div>
    <h1 style="color: white">Welcome to SapShell</h1>
</div>
<button class="sign-in-btn" onclick="signOut()">Sign In</button>
    <script>
        function searchRedirect1() {
            google.colab.kernel.invokeFunction('notebook.Redirect1', [], {}).then(function(response){}).catch(function(error){console.error(error);});
        }
        function searchRedirect2() {
            google.colab.kernel.invokeFunction('notebook.Redirect2', [], {}).then(function(response){}).catch(function(error){console.error(error);});
        }
        function searchRedirect3() {
            google.colab.kernel.invokeFunction('notebook.Redirect3', [], {}).then(function(response){}).catch(function(error){console.error(error);});
        }
        function signOut() {
        // Show an alert message to the user
        localStorage.removeItem("username");
        localStorage.clear();
        alert("Signing out...");
        // Redirect to the homepage after showing the alert
        google.colab.kernel.invokeFunction('notebook.Redirect4', [], {}).then(function(response){}).catch(function(error){console.error(error);});

    }
    </script>
</body>
</html>

"""

html_code_initial_user = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SapShell</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            height: 100vh;
            margin: 0;
            background-color: #0c2d40;
        }
        .search-container {
            text-align: center;
        }
      .sidebar {
            width: 200px;
            background-color: #e6e8ed;
            color: #0c2d40;
            height: 100vh;
            position: fixed;
            left: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }
       .sidebar h2, .sidebar a, .sidebar p {
            margin-top: 10px;
            margin-bottom: 20px; /* Increased bottom margin for more space */
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            display: block; /* Ensure full width for easier clicking */
        }
        .sidebar a:hover, .sidebar p:hover {
            background-color: #0c2d40;
            color: #e6e8ed;
            cursor: pointer;
        }
        .sidebar a, .sidebar p {
            text-decoration: none;
            color: #0c2d40;
        }
         .sign-in-btn {
            position: fixed;
            bottom: 20px;
            right: 20px;
            padding: 10px 20px;
            background-color: #e6e8ed;
            color: #0c2d40;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }
        .sign-in-btn:hover {
            background-color: #0c2d40;
            color: #e6e8ed;
        }
    </style>
</head>
<body>
<div class="sidebar">
    <div>
        <h2>Navigation</h2>
        <ul>
            <a id="searchButton">Search</a>
            <a id="searchHistoryButton">Show Search History</a>
            <div>
                <p class="sign-out" onclick="signOut()">🚪 Sign out</p>
            </div>
        </ul>
    </div>
</div>
<div class="search-container">
    <div class="logo">
        <img width="400" height="200" src="https://firebasestorage.googleapis.com/v0/b/sapshell-cloud.appspot.com/o/turtleslogo2.jpg?alt=media&token=7eb87c27-4ae2-4cac-97e8-c20378730723" alt="SAPSHELL">
    </div>
    <h1 style="color: white">Welcome to SapShell</h1>
</div>
<button class="sign-in-btn" onclick="signOut()">Sign In</button>
<script>
        var user = localStorage.getItem("username"); // Inject the Python value of `user` into JavaScript
        document.getElementById('searchHistoryButton').addEventListener('click', function() {
              google.colab.kernel.invokeFunction('notebook.display_search_history', [user], {})
        });
      document.getElementById('searchButton').addEventListener('click', function() {
       google.colab.kernel.invokeFunction('notebook.Redirect1', [], {})
});

        function signOut() {
        // Show an alert message to the user
          localStorage.removeItem("username");
          localStorage.clear();
          sessionStorage.clear();
          alert("Signing out...");

        // Redirect to the homepage after showing the alert
        google.colab.kernel.invokeFunction('notebook.Redirect4', [], {}).then(function(response){}).catch(function(error){console.error(error);});

    }
    </script>
</body>
</html>

"""



# Display the initial HTML in the output cell
display(HTML(html_code_initial))

# Callbacks registration
output.register_callback('notebook.Redirect1', redirect_to_results)
output.register_callback('notebook.Redirect2', redirect_to_index_manager)
output.register_callback('notebook.Redirect3', redirect_to_statistics)
output.register_callback('notebook.Redirect4', redirect_to_homepage)
output.register_callback('notebook.display_search_history', display_search_history)
output.register_callback('notebook.FetchRareWordData', fetch_and_process_data_for_chart())
output.register_callback('notebook.redirectToLogin', redirectToLogin)
output.register_callback('notebook.delete_index', delete_index)
output.register_callback('notebook.search_result', search_index)
output.register_callback('notebook.pagination', pagination)
output.register_callback('notebook.loginuser', loginuser)
output.register_callback('notebook.loginadmin', loginuser)
output.register_callback('notebook.create_account', create_account)
output.register_callback('notebook.add_index_in_db', add_index_in_db)